In [1]:
import ramanchada2 as rc2
import ramanchada2.misc.constants  as rc2const
import ramanchada2.misc.utils as rc2utils
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def diff(spe, ref_pos):
    ss = spe.subtract_moving_minimum(40)
    kw = dict(sharpening=None, hht_chain=[100])
    cand = ss.find_peak_multipeak(**kw)
    kw = dict(profile='Gaussian')
    fit_res = spe.fit_peak_multimodel(candidates=cand, **kw)

    spe_pos, spe_pos_err = fit_res.centers_err.T

    spe_pos_match_idx, ref_pos_match_idx = rc2utils.find_closest_pairs_idx(spe_pos, ref_pos)
    spe_pos_match = spe_pos[spe_pos_match_idx]
    ref_pos_match = ref_pos[ref_pos_match_idx]
    return ref_pos_match, (spe_pos_match-ref_pos_match), spe_pos_err[spe_pos_match_idx]

In [3]:
spe_nocalib = rc2.spectrum.from_local_file('data/2 - Full Data sets/FMNT-M_Ho785/NeonSNQ043_iR785_OP03.txt', filetype='txt')

FileNotFoundError: [Errno 2] No such file or directory: 'data/2 - Full Data sets/FMNT-M_Ho785/NeonSNQ043_iR785_OP03.txt'

In [ ]:
neon_pos_wl_dict = rc2const.neon_wl_spe[785].fit_peak_positions(center_err_threshold=.1)
neon_pos_wl = np.array(list(neon_pos_wl_dict.keys()))

neon_pos_rs_dict = rc2utils.abs_nm_to_shift_cm_1_dict(neon_pos_wl_dict, 785)
neon_pos_rs = np.array(list(neon_pos_rs_dict.keys()))

In [ ]:
spe_cal = [spe_nocalib.shift_cm_1_to_abs_nm_filter(785)]
for _ in range(10):
    spe_cal.append(spe_cal[-1].xcal_fine(ref=neon_pos_wl_dict , poly_order=3))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5), nrows=1, sharex=True, tight_layout=True)

ax.axhline(0)
ax.set_title('FMNT-M_Ho785/NeonSNQ043_iR785_OP03')
ax.set_xlabel('Raman shift [cm$^{-1}$]')
ax.set_ylabel('Δ shift [cm$^{-1}$]')

for spe_i, spe in enumerate(spe_cal):
    ax.errorbar(*diff(spe.abs_nm_to_shift_cm_1_filter(785), neon_pos_rs), fmt='.:', label=spe_i)
ax.legend()
ax.grid()
ax.set_ylim(-8, 15)


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5), nrows=1, sharex=True, tight_layout=True)

ax.axhline(0)
ax.set_title('FMNT-M_Ho785/NeonSNQ043_iR785_OP03')
ax.set_xlabel('Raman shift [cm$^{-1}$]')
rc2const.neon_rs_spe[785].plot(ax=ax.twinx(), fmt='r:')
for spe_i, spe in enumerate(spe_cal):
    spe.abs_nm_to_shift_cm_1_filter(785).plot(ax=ax, fmt=':', label=spe_i)

ax.set_xlim(200, 2000)
ax.legend()
ax.grid()